# Introduction :

Dans ce projet, je me concentre sur deux tâches principales : l'extraction des entités et la détection des signatures dans des documents PDF, spécifiquement pour des modèles de contrats d'assurance.

L'objectif est d'extraire les informations essentielles contenues dans ces documents, telles que le nom de l'assuré, l'adresse, la date d'effet, le Numéro d'assurance, le Montant de la Couverture et la Réduction ainsi que d'identifier si le document est signé par les deux parties concernées.

Ce traitement est effectué sur des documents structurés selon un modèle spécifique (le PDF fourni), où les informations clés sont souvent répétées et placées à des positions précises dans le texte. Pour cela, j'utilise des techniques de reconnaissance de texte et d'analyse d'images, permettant d'automatiser l'extraction des données importantes et la détection des signatures après des mots-clés comme "Yours sincerely", typiques de la conclusion d'un contrat.

# Installation

In [ ]:
pip install PyPDF2 pdfplumber spacy


# 1) Extraction des Entités :

In [ ]:
import pdfplumber
import re
import csv

### Fonction pour extraire le Nom et l'Adresse de l'assuré :

Cette Fonction extrait le nom et l'adresse d'un assuré à partir d'un texte brut issu d'un PDF. Elle repose sur l'hypothèse que tous les PDF suivent un format fixe où :

1) Le nom de l'assuré est toujours précédé de "Mr" ou "Ms".

2) L'adresse commence immédiatement après la ligne contenant le nom et se termine par une ligne contenant un code postal à 4 chiffres suivi d'une ville.

La fonction parcourt ligne par ligne le texte, détecte le nom via un motif regex, puis capture les lignes suivantes comme étant l'adresse jusqu'à la détection du code postal. Grâce à cette hypothèse de format constant, elle garantit une extraction cohérente et précise des informations.

In [111]:
import re

def extract_name_and_address(text):
    # Diviser le texte en lignes en utilisant le caractère de nouvelle ligne "\n"
    lines = text.split("\n")

    # Définition d'un motif(pattern) pour détecter un nom précédé de "Mr" ou "Ms" suivi d'un ou plusieurs noms.
    # Supposition : Le texte suit toujours le même modèle, où le nom est précédé par "Mr" ou "Ms".
    name_pattern = r"^(Mr|Ms)\s+([A-Za-z\s]+)$"

    # Définition d'un motif pour détecter un code postal à 4 chiffres suivi d'une ville
    postal_code_pattern = r"^\d{4}\s+[A-Za-z\s]+$"

    # Initialisation des variables pour stocker le nom et les lignes d'adresse
    name, address_lines = None, []

    # Indicateur pour savoir si on doit commencer à capturer l'adresse
    is_address = False

    # Boucle sur chaque ligne extraite du texte
    for line in lines:
        # Si la ligne correspond au motif du nom (nom de l'assuré), on la stocke
        match = re.match(name_pattern, line.strip())
        if match:
            # Si le titre est "Mr" ou "Ms", on extrait uniquement la partie du nom (sans le titre)
            name = match.group(2).strip()  # Enlève les espaces en début/fin de ligne et stocke le nom sans "Mr" ou "Ms"
            is_address = True    # Active la capture des lignes suivantes pour l'adresse

        # Si l'indicateur "is_address" est activé, on commence à traiter l'adresse
        elif is_address:
            # Vérifie si la ligne correspond au motif du code postal (fin de l'adresse)
            if re.match(postal_code_pattern, line.strip()):
                address_lines.append(line.strip())  # Ajoute la ligne de l'adresse
                break  # Arrête la capture après avoir trouvé la ligne de code postal
            # Si la ligne n'est pas vide, elle est ajoutée comme partie de l'adresse
            elif line.strip():
                address_lines.append(line.strip())

    # Concatène les lignes d'adresse en une seule chaîne ou indique "Non trouvé" si vide
    address = " ".join(address_lines) if address_lines else "Non trouvé"

    # Retourne un dictionnaire contenant le nom et l'adresse de l'assuré
    return {"Nom de l'assuré": name, "Adresse de l'assuré": address}


### Fonction pour extraire la date d'effet :

Cette fonction extrait la date d'effet, c'est-à-dire la première date mentionnée dans le texte qui correspond au format "Jour Mois Année" ( "17 January 2024").


1) Diviser le texte en lignes : Le texte est d'abord divisé en lignes pour faciliter la recherche.

2) Définir un motif de recherche pour la date d'effet: (expression régulière) est utilisé pour rechercher une date dans le format Jour Mois Année.

3) Retour de la date d'effet : Si une date est trouvée, elle est renvoyée comme date d'effet. Si aucune date n'est trouvée, la fonction renvoie "Non trouvée".

In [112]:
def extract_date(text):
    # Diviser le texte en lignes
    lines = text.split("\n")

    # Définir un motif pour détecter une date au format : Jour Mois Année ( "17 January 2024")
    date_pattern = r"\b\d{1,2}\s+(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b"

    # Initialiser la variable de la date
    extracted_date = None

    # Parcourir chaque ligne pour rechercher une date
    for line in lines:
        match = re.search(date_pattern, line.strip())
        if match:
            extracted_date = match.group(0)
            break  # Arrêter dès qu'une date est trouvée

    # Retourner la date ou "Non trouvée"
    return {"Date": extracted_date if extracted_date else "Non trouvée"}

### Fonction pour extraire le Numéro d'assurance

Cette fonction extrait le numéro d'assurance à partir d'un texte brut :

1) Format fixe : La fonction repose sur l'hypothèse que tous les PDF suivent un format où le numéro d'assurance est précédé de l'expression "insurance no.".

2) Utilisation de regex : Un motif regex est utilisé pour repérer "insurance no." suivi d'un ou plusieurs chiffres et espaces.

3)Résultat : Si une correspondance est trouvée, le numéro est extrait et les espaces superflus sont supprimés. Sinon, la fonction retourne "Non trouvé".



In [113]:
def extract_insurance_number(text):
    match = re.search(r"insurance no\.\s+([\d\s]+)", text)
    return match.group(1).strip() if match else "Non trouvé"

# Fonction pour extraire le Montant de la Couverture

Cette fonction extrait le montant de la couverture (prime mensuelle totale en CHF) :

1) Hypothèse de format fixe : Le texte suit toujours un modèle où la phrase "Total monthly premium in CHF payable by you" est suivie du montant sous forme numérique avec deux décimales (ex. : 1234.56).

2) Utilisation de regex : La fonction utilise un motif regex pour repérer cette phrase et extraire le montant correspondant.

3) Résultat :
Si le montant est trouvé, il est directement retourné.
Si aucune correspondance n'est détectée, la fonction retourne "Non trouvé".

In [114]:
def extract_coverage_amount(text):
    match = re.search(r"Total monthly premium in CHF payable by you\s+(\d+\.\d{2})", text)
    return match.group(1) if match else "Non trouvé"

# Fonction pour extraire la Réduction

Cette fonction extrait toutes la réduction Totale :

1) Hypothèse de format : La réduction est indiquée sous la forme "total discount ... CHF [montant]".

2) Extraction avec regex :
Le motif regex détecte les réductions sous forme numérique (ex. : 150.00). Le montant extrait est nettoyé des espaces superflus.

3) Résultat : La fonction retourne la réduction Totale.


In [115]:
def extract_discount_from_all_pages(pdf):
    all_discount = []  # stocker toutes les réductions extraites
    for page in pdf.pages:
        text = page.extract_text()
        match_reduction = re.search(r"total discount.*?CHF\s([\d\.]+)", text, re.IGNORECASE)
        if match_reduction:
            reduction = match_reduction.group(1).strip()
            if reduction not in all_discount:  # Eviter les doublons
                all_discount.append(reduction)
    return all_discount

# Fonction pour extraire le Nom de l'Assureur

Cette fonction extrait le nom de l'assureur à partir du pied de la page :

1) Hypothèse de format : Le nom de l'assureur est toujours situé dans les quatre dernières lignes du texte, généralement dans le pied de page.

2) Extraction des lignes du pied de page : La fonction sélectionne les quatre dernières lignes pour concentrer la recherche.

3) Recherche par regex :
Le motif regex détecte les noms propres (mots commençant par une majuscule suivis de lettres minuscules, avec des espaces optionnels entre eux).
Les lignes contenant des URLs, des extensions (.com, .ch) ou des chiffres sont exclues pour éviter de fausses correspondances.

4) Résultat :
Si un nom correspondant est trouvé, il est retourné après suppression des espaces superflus.
Si aucun assureur n'est détecté, la fonction retourne "Non trouvé".

In [116]:
def extract_insurer_from_footer(text):
    lines = text.split("\n")
    footer_lines = lines[-4:]  # Dernières lignes de la page
    potential_insurer = None

    for line in footer_lines:
        # Recherche de noms propres
        if re.search(r"[A-Z][a-z]+(?:\s[A-Z][a-z]+)*", line) and not re.search(r"(www\.|\.com|\.ch|\d+)", line):
            potential_insurer = line.strip()
            break

    return potential_insurer if potential_insurer else "Non trouvé"

# Test  :

In [117]:
# Extraction des entités du document
pdf_path = "/content/insurance-policy-example.pdf"
with pdfplumber.open(pdf_path) as pdf:
    text_first_page = pdf.pages[0].extract_text()

    extracted_info = {}
    extracted_info["Nom de l'assuré"], extracted_info["Adresse de l'assuré"] = extract_name_and_address(text_first_page).values()
    extracted_info["Numéro d'assurance"] = extract_insurance_number(text_first_page)
    extracted_info["Montant de la couverture"] = extract_coverage_amount(text_first_page)
    extracted_info["Nom de l'assureur"] = extract_insurer_from_footer(text_first_page)
    extracted_info["Date"] = extract_date(text_first_page)["Date"]
    all_discount = extract_discount_from_all_pages(pdf)
    extracted_info["Réduction"] = ", ".join(all_discount) if all_discount else "Non trouvé"

# Affichage des résultats
for key, value in extracted_info.items():
    print(f"{key}: {value if value else 'Non trouvé'}")

Nom de l'assuré: Charles Muster
Adresse de l'assuré: Feldlerchenweg 15 3360 Herzogenbuchsee
Numéro d'assurance: 100 452 956
Montant de la couverture: 863.55
Nom de l'assureur: Helsana Versicherungen AG
Date: 17 January 2024
Réduction: 7.50.


# Stocker les données extraites dans un Fichier CSV

In [118]:
csv_file_path = "/content/extracted_info.csv"
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=extracted_info.keys())

    # Écriture de l'en-tête
    writer.writeheader()

    # Écriture des données extraites
    writer.writerow(extracted_info)

print(f"Les informations extraites ont été enregistrées dans {csv_file_path}")

Les informations extraites ont été enregistrées dans /content/extracted_info.csv


# 2) Détection de Signatures :

1) Conversion de la dernière page du PDF en image : La première étape consiste à convertir la dernière page du PDF en image, car cette page contient généralement les signatures.

2) Localisation du mot-clé "Yours sincerely" : Ensuite, le mot-clé "Yours sincerely" est localisé dans l'image, car les signatures se trouvent systématiquement sous ce mot.

3) Extraction des signatures : Une fois le mot-clé localisé, une extraction des signatures peut être réalisée en identifiant et capturant la zone sous ce mot dans l'image. Ce processus permet de détecter les régions où les signatures sont présentes.

4) Extraction des noms après "Yours sincerely" : Après avoir localisé et extrait les signatures, le texte du PDF est analysé pour en extraire les noms présents après le mot-clé "Yours sincerely". Cela permet de récupérer les noms de toutes les personnes signant le document, y compris les noms des signataires et de l'assuré.

5) Comparaison des noms extraits avec le nom de l'assuré : Si le nom de l'assuré est trouvé parmi les signataires, cela confirme que l'assuré a bien signé le document.

6) Vérification de la présence de deux signatures : Enfin, la fonction vérifie que le document contient au moins deux signatures détectées sous le mot-clé "Yours sincerely". Si le nom de l'assuré est trouvé et que deux signatures sont présentes, cela confirme que le document est signé par les deux parties (y compris l'assuré).

## Installation :

In [ ]:
!apt-get update
!apt-get install -y poppler-utils


!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,619 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,513 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,5

In [ ]:
!pip install pytesseract


In [ ]:
!pip install pdf2image


In [ ]:
import os
import cv2
import pytesseract
from pdf2image import convert_from_path
import re

In [119]:
# Je vais utiliser pytesseract pour extraire du texte à partir d'images.
# Tesseract est un moteur de reconnaissance optique de caractères (OCR) qui est utilisé pour extraire
# du texte à partir d'images. La bibliothèque pytesseract permet d'interagir avec Tesseract depuis Python,
# mais elle a besoin de connaître l'emplacement de l'exécutable de Tesseract pour fonctionner correctement.
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'


# Fonction de conversion de la dernière page du PDF en image

Cette fonction convertit la dernière page d'un fichier PDF en une image PNG. Elle utilise la bibliothèque pdf2image pour convertir le PDF en images et enregistre l'image de la dernière page dans un dossier de sortie. Le chemin de l'image enregistrée est ensuite retourné.

In [120]:
def convert_last_page_to_image(pdf_path, output_folder="output_image"):
    images = convert_from_path(pdf_path)
    os.makedirs(output_folder, exist_ok=True)
    last_image = images[-1]
    image_path = os.path.join(output_folder, "last_page.png")
    last_image.save(image_path, "PNG")
    return image_path

# Fonction de recherche du mot-clé dans l'image

Cette fonction permet de trouver la position de mot-clé: "Yours sincerely" dans l'image en utilisant l'OCR avec Tesseract.

La fonction traite l'image, effectue l'OCR, et identifie la position du mot-clé dans l'image, en tenant compte du fait que le mot-clé pourrait être séparé en deux parties ("Yours" et "sincerely").



In [121]:
def find_keyword_position(image_path, keyword="Yours sincerely"):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    data = pytesseract.image_to_data(gray, lang="eng", output_type=pytesseract.Output.DICT)
    keyword_parts = keyword.lower().split()
    keyword_positions = []
    current_position = None

    for i, word in enumerate(data["text"]):
        if word.lower() == keyword_parts[0] and current_position is None:  # Première partie "Yours"
            current_position = (data["left"][i], data["top"][i], data["width"][i], data["height"][i])
        if word.lower() == keyword_parts[1] and current_position:  # Deuxième partie "sincerely"
            x, y, w, h = data["left"][i], data["top"][i], data["width"][i], data["height"][i]
            keyword_positions.append((current_position[0], current_position[1], w, h))
            break

    return keyword_positions[0] if keyword_positions else None


# Détection des signatures après le mot-clé

Cette fonction détecte les régions de signature dans une image, mais uniquement sous le mot-clé spécifié (par défaut "Yours sincerely").

L'image est d'abord convertie en niveaux de gris et traitée pour détecter les contours. Ensuite, les régions en dessous du mot-clé sont extraites et enregistrées sous forme d'images.

Les contours sont filtrés en fonction de leur taille pour ne conserver que ceux qui correspondent à des signatures.



In [122]:
def detect_signatures_after_keyword(image_path, keyword_position, output_folder="signatures"):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    signature_regions = []
    os.makedirs(output_folder, exist_ok=True)
    keyword_y = keyword_position[1] if keyword_position else 0

    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if 500 < area < 10000:
            x, y, w, h = cv2.boundingRect(contour)
            if y > keyword_y:
                signature_regions.append((x, y, w, h))
                roi = image[y:y+h, x:x+w]
                region_path = os.path.join(output_folder, f"signature_{i + 1}.png")
                cv2.imwrite(region_path, roi)

    return signature_regions


# Extraction des Noms des Signataires

Cette fonction extrait les noms situés après l'expression "Yours sincerely" dans un texte donné. Elle recherche les lignes contenant les noms et les nettoie des mots-clés tels que "CEO" ou "Head".

 Ensuite, la fonction normalize_name est utilisée pour normaliser les noms extraits en les convertissant en minuscules et en supprimant les espaces inutiles.

 La fonction check_name_in_extracted_list compare le nom de l'assuré, après l'avoir normalisé, avec les noms extraits pour vérifier si tous les mots du nom de l'assuré apparaissent dans les noms extraits.

In [123]:
# Extraction des noms après "Yours sincerely"
def extract_names_after_sincerely(text):
    pattern = r"Yours sincerely\s*(.*?)(?:\n|\r)(.*?)(?:\n|\r)"
    match = re.search(pattern, text, re.DOTALL)

    if match:
        name_line_1 = match.group(1).strip()
        name_line_2 = match.group(2).strip()

        names = []
        for line in [name_line_1, name_line_2]:
            if not any(word in line for word in ["CEO", "Head"]):
                names.append(line)

        return names
    return []

# Fonction pour normaliser les noms
def normalize_name(name):
    return name.strip().lower()

# Fonction pour vérifier si tous les mots du nom de l'assuré sont dans les noms extraits
def check_name_in_extracted_list(insured_name, extracted_names):
    # Normaliser le nom de l'assuré
    normalized_insured_name = normalize_name(insured_name)
    insured_name_parts = normalized_insured_name.split()

    # Vérifier si tous les mots de l'assuré sont présents dans les noms extraits
    for name in extracted_names:
        normalized_extracted_name = normalize_name(name)
        if all(part in normalized_extracted_name for part in insured_name_parts):
            return True  # Si tous les mots sont trouvés dans ce nom
    return False  # Si aucun nom n'est trouvé avec tous les mots

## Test

In [124]:
# Fonction principale pour traiter la dernière page et extraire les signatures et noms
def process_last_page_for_signatures_after_keyword(pdf_path, insured_name, keyword="Yours sincerely"):
    last_page_image = convert_last_page_to_image(pdf_path)
    keyword_position = find_keyword_position(last_page_image, keyword)

    if keyword_position:
        # Extraire les régions de signature
        signature_regions = detect_signatures_after_keyword(last_page_image, keyword_position)

        # Extraire le texte du PDF pour les noms
        with pdfplumber.open(pdf_path) as pdf:
            text = "\n".join(page.extract_text() for page in pdf.pages)

        # Extraire les noms après "Yours sincerely"
        extracted_names = extract_names_after_sincerely(text)

        # print(f"Extracted names from the document: {extracted_names}")

        # Vérifier si le nom de l'assuré est dans la liste des noms extraits
        is_name_found = check_name_in_extracted_list(insured_name, extracted_names)

        # Si le nom de l'assuré est trouvé dans la liste et qu'il y a plus de 2 signatures
        if is_name_found and len(signature_regions) >= 2:
            print("The document is signed by both parties.")
        else:
            print("The document is not signed by both parties.")
    else:
        print(f"Keyword '{keyword}' not found in the last page.")

In [127]:
pdf_path = "/content/insurance-policy-example.pdf"

with pdfplumber.open(pdf_path) as pdf:
    text_first_page = pdf.pages[0].extract_text()

    # Extraction du nom de l'assuré
    insured_name = extract_name_and_address(text_first_page)["Nom de l'assuré"]
    print(f"Insured's name extracted: {insured_name}")


process_last_page_for_signatures_after_keyword(pdf_path, insured_name, keyword="Yours sincerely")

Insured's name extracted: Charles Muster
The document is not signed by both parties.


# Conclusion

Dans ce projet, j'ai utilisé des fonctions et des techniques spécifiques basées sur le modèle de contrat fourni pour extraire les entités et détecter les signatures. Cependant, si plusieurs modèles de contrats existent, il serait possible d'utiliser des systèmes LLMs (Large Language Models) associés à des mécanismes RAG (Retrieval-Augmented Generation). Ces systèmes peuvent analyser l'intégralité du texte, comprendre les informations présentes et, à la fin, générer automatiquement les entités souhaitées, indépendamment des variations de format ou de structure des documents.